In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'book-recommendation-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1004280%2F7595263%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240306%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240306T131243Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4966aaf30f8a31b928b600f2d78f129326bbfd15d6c6cfbd9e07261ee532af1cfc6566ebf94f47ad7accde059418dd0bdc01056cbce33d9b016f6e9e4329e3dd1704276de8f352ea3e5ee217e70d27ecb5bc39918c0386ce57c07f0daf87b9736bf01f247b28290a1e3e1c9e142e2e65602a4314872d4de566f507610bfdc39fe59710322fc1ccaaf22200f4d027bd678873626580f408af9f6fdaae7d395a894e36dc3f97242f5bcdfae3b1dd188201feb539441278c949f2636af4b82dea9970a83a784518f9263970481dc2ec76a597b489b836d21043eabe2e88c0a5eabf8a309ce39c35b3ea8017e30192983627c78e2af1334ca67778500ba4259c8842'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 25500788 bytes downloaded
Downloaded and uncompressed: book-recommendation-dataset
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-dataset/Books.csv
/kaggle/input/book-recommendation-dataset/Ratings.csv
/kaggle/input/book-recommendation-dataset/classicRec.png
/kaggle/input/book-recommendation-dataset/DeepRec.png
/kaggle/input/book-recommendation-dataset/Users.csv
/kaggle/input/book-recommendation-dataset/recsys_taxonomy2.png


In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import re
from PIL import Image
import requests
import random
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

In [4]:
book = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv")
rating = pd.read_csv("/kaggle/input/book-recommendation-dataset/Ratings.csv")
user = pd.read_csv("/kaggle/input/book-recommendation-dataset/Users.csv")

In [5]:
book.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [6]:
user.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [7]:
rating.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

## Popularity Based

In [8]:

df=book.merge(rating,on="ISBN")
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [9]:
df=df.copy()
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.drop(columns=["ISBN","Year-Of-Publication","Image-URL-S","Image-URL-M"],axis=1,inplace=True)
df.drop(index=df[df["Book-Rating"]==0].index,inplace=True)

In [10]:
df

,Book-Title,Book-Author,Publisher,Image-URL-L,User-ID,Book-Rating
1,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,8,5
3,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,11676,8
5,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,67544,8
8,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,116866,9
9,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,123629,9
...,...,...,...,...,...,...
1031116,Dreamsnake,Vonda N. McIntyre,Houghton Mifflin,http://images.amazon.com/images/P/0395264707.0...,275318,10
1031121,Cocktail Classics,David Biggs,Connaught,http://images.amazon.com/images/P/1845170423.0...,275970,7
1031123,Flashpoints: Promise and Peril in a New World,Robin Wright,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,276313,5
1031124,There's a Bat in Bunk Five,Paula Danziger,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,276463,7


In [12]:
def popular_books(df, n=100):
    rating_count = df.groupby("Book-Title").agg({'Book-Rating': 'count', 'Image-URL-L': 'first'}).reset_index()
    rating_count.rename(columns={"Book-Rating": "NumberOfVotes"}, inplace=True)

    rating_average = df.groupby("Book-Title").agg({'Book-Rating': 'mean'}).reset_index()
    rating_average.rename(columns={"Book-Rating": "AverageRatings"}, inplace=True)

    popularBooks = rating_count.merge(rating_average, on="Book-Title")

    def weighted_rate(x):
        v = x["NumberOfVotes"]
        R = x["AverageRatings"]
        return ((v * R) + (m * C)) / (v + m)

    C = popularBooks["AverageRatings"].mean()
    m = popularBooks["NumberOfVotes"].quantile(0.90)

    popularBooks = popularBooks[popularBooks["NumberOfVotes"] >= 250]
    popularBooks["Popularity"] = popularBooks.apply(weighted_rate, axis=1)
    popularBooks = popularBooks.sort_values(by="Popularity", ascending=False)
    return popularBooks[["Book-Title", "NumberOfVotes", "AverageRatings", "Popularity", "Image-URL-L"]].reset_index(drop=True).head(n)


In [13]:
import pandas as pd

# Assuming 'popular_books' is a function that returns the top N popular books from 'df'
n = 10
top_ten = pd.DataFrame(popular_books(df, n))

# Prepare the display information
print("MOST POPULAR 10 BOOKS\n")
for i, row in top_ten.iterrows():
    book_title = row["Book-Title"]
    image = row["Image-URL-L"]
    average_rating = round(df[df["Book-Title"] == book_title]["Book-Rating"].mean(), 1)
    print(f"{i+1}. {book_title} - Rating: {average_rating} -Link:{image}")


MOST POPULAR 10 BOOKS

1. Harry Potter and the Prisoner of Azkaban (Book 3) - Rating: 9.0 -Link:http://images.amazon.com/images/P/0439136350.01.LZZZZZZZ.jpg
2. To Kill a Mockingbird - Rating: 9.0 -Link:http://images.amazon.com/images/P/0446310786.01.LZZZZZZZ.jpg
3. Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)) - Rating: 8.9 -Link:http://images.amazon.com/images/P/059035342X.01.LZZZZZZZ.jpg
4. Harry Potter and the Chamber of Secrets (Book 2) - Rating: 8.8 -Link:http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg
5. Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson - Rating: 8.6 -Link:http://images.amazon.com/images/P/0385484518.01.LZZZZZZZ.jpg
6. The Secret Life of Bees - Rating: 8.5 -Link:http://images.amazon.com/images/P/0142001740.01.LZZZZZZZ.jpg
7. The Da Vinci Code - Rating: 8.4 -Link:http://images.amazon.com/images/P/0385504209.01.LZZZZZZZ.jpg
8. The Lovely Bones: A Novel - Rating: 8.2 -Link:http://images.amazon.com/images/P/0316

## Matrix Factorisation

In [15]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162998 sha256=8738722be2bdbbbfc06facb4cde7033541b71789e1b8ff57146ab1ecfe111cf7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [16]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

def matrix_factorization_recommendation(df, user_id, n=10):

    reader = Reader(rating_scale=(1, 10))
    data = Dataset.load_from_df(df[['User-ID', 'Book-Title', 'Book-Rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    svd = SVD()
    svd.fit(trainset)

    user_unrated_books = df.loc[~df['Book-Title'].isin(df[df['User-ID'] == user_id]['Book-Title'].unique())]['Book-Title']

    user_testset = [(user_id, book_title, 0) for book_title in user_unrated_books]
    user_predictions = svd.test(user_testset)

    pred_df = pd.DataFrame(user_predictions, columns=['User-ID', 'Book-Title', 'Actual-Rating', 'Predicted-Rating', 'Details'])
    pred_df = pred_df.sort_values(by='Predicted-Rating', ascending=False)

    top_books = pred_df[['Book-Title', 'Predicted-Rating']].drop_duplicates().head(n)

    return top_books




In [17]:

user_id_input = input("Enter a User-ID for personalized recommendations: ")
user_id_input = int(user_id_input)
top_books_user = matrix_factorization_recommendation(df, user_id_input)
print(top_books_user)

Enter a User-ID for personalized recommendations: 276769
                                               Book-Title  Predicted-Rating
183650       My Sister's Keeper : A Novel (Picoult, Jodi)          9.099899
261288  Harry Potter and the Chamber of Secrets Postca...          9.099011
134318                                  Calvin and Hobbes          9.064295
242343                       Dilbert: A Book of Postcards          9.012598
86801                               84 Charing Cross Road          8.986016
18450      The Two Towers (The Lord of the Rings, Part 2)          8.976523
130878                                          Illusions          8.944884
54416                                          The Hobbit          8.937315
40983       Ender's Game (Ender Wiggins Saga (Paperback))          8.899593
26449                                       Lonesome Dove          8.899173
